In [ ]:
#load data
library(data.table)
library(sp)
library(rgdal)
library(rgeos)
library(animation)
nine11 = fread('Seattle_Real_Time_Fire_911_Calls.csv')
#eliminate calls missing geospatial tags
nine11 = nine11[!is.na(Longitude)]
nine11[ , Datetime :=
          as.POSIXct(Datetime, format = '%m/%d/%Y %I:%M:%S %p +0000')]
#a relatively small number of rows have mal-formatted Datetime ala
#  YYYY-MM-DDT24:MM:SS+0000 (MM >= 0) perhaps means after midnight?
nine11 = nine11[!is.na(Datetime)]
nine11[ , yr := year(Datetime)]
nine11[ , wk := week(Datetime)]
#assign pd_no to skip grouping by .(yr, wk) all the time
nine11[order(yr, wk), pd_no := .GRP, by = .(yr, wk)]
setkey(nine11, pd_no)
#use type coding from here:
#  http://www2.seattle.gov/fire/incidentSearch/TypeCode.htm
#  done by hand since the match is inexact and there's few enough
#  not to warrant fuzzy matching; should revisit as it appears
#  some codes have been updated since 2013 when website was updated
type_table = fread('type_category_match.csv')
#join to table
nine11[type_table, category := i.category, on = 'Type']
#subset to well-represented categories
nine11 = nine11[category %in% c('fire', 'aid', 'medic')]
#convert to SpatialPoints to spatially join
nine11SP = SpatialPointsDataFrame(
  nine11[ , cbind(Longitude, Latitude)], data = nine11,
  proj4string = CRS('+init=epsg:4326')
)
#neighborhoods shapefile from
#  https://data.seattle.gov/dataset/Neighborhoods/2mbt-aqqx
seattle = readOGR('.', 'Neighborhoods')
#convert to data.table for easier join syntax
seattle@data = setDT(seattle@data)
#re-cast coordinates for spatial join
nine11SP = spTransform(nine11SP, proj4string(seattle))
#spatial join, assign neighborhood to each crime
nine11SP$nbhd = (nine11SP %over% seattle)$OBJECTID
#re-record to data.table
nine11[ , nbhd := nine11SP$nbhd]
#eliminate outlier crimes not in Seattle boundary
seattle_idx = !is.na(nine11SP$nbhd)
nine11 = nine11[seattle_idx]
nine11SP = nine11SP[seattle_idx, ]
# PLOT 1 - GIF of hotspots by neighborhood
## add columns with counts by week/neighborhood
##   to SPDF to facilitate plotting
seattle@data = 
  seattle@data[dcast(nine11, nbhd ~ category + pd_no, 
                     fun.aggregate = length, value.var = 'Type'),
               on = c(OBJECTID = 'nbhd')]
#cuts chosen with nice cutoffs to reflect relative intensity
cols = colorRampPalette(c('white', 'red'))(6L)
col_cuts = c(0, 1, 11, 26, 51, 101)
colorize = function(nn) cols[findInterval(nn, col_cuts)]
#for human-parseable version of period #
pretty_date = 
  nine11[ , .(full_date = format(min(Datetime), '%B %d, %Y')), 
          keyby = .(pd_no, yr, wk)]
plot_pd = function(period, categ)
  plot(seattle, col = colorize(seattle[[paste0(categ, '_', period)]]),
       main = paste0('Fire Alerts to 911 in Seattle\n',
                     'Week of ', pretty_date[.(period), full_date]))
#pick all the period numbers in the past year
past_year =
  pretty_date[pd_no >= pd_no[yr == 2016 & wk == week(Sys.Date())], pd_no]
saveGIF(for (ii in past_year) plot_pd(ii, 'fire'), 'fire_911.gif')
# PLOT 2 - smoothed scatterplots of autocorrelation
png('fire_911_lags.png', width = 960, height = 960)
par(mfrow = c(1, 2), oma = c(0, 0, 2, 0))
nine11[category == 'aid', .N, keyby = .(nbhd, pd_no)
       ][ , lag_N := shift(N, type = 'lead'), by = nbhd
          #N outside [0, 20] were distracting from the main takeaway
          ][N<20 & lag_N<20, {
            smoothScatter(N, lag_N, main = 'One-Week Lag',
                          xlab = '# Calls Last Week',
                          ylab = '# Calls This Week',
                          colramp = colorRampPalette(c('white', 'red')))
          }]
nine11[category == 'aid', .N, keyby = .(nbhd, pd_no)
       ][ , lag_N := shift(N, n = 52L, type = 'lead'), by = nbhd
          #N outside [0, 20] were distracting from the main takeaway
          ][N<20 & lag_N<20, {
            smoothScatter(N, lag_N, main = 'One-Week Lag',
                          xlab = '# Calls This Week',
                          ylab = '# Calls This Week Last Year',
                          colramp = colorRampPalette(c('white', 'red')))
          }]
title('Serial Dependence of 911 Fire Calls in a Neighborhood', outer = TRUE, cex.main = 2)
dev.off()